<a href="https://colab.research.google.com/github/douglasngomes/Project2201/blob/main/PROJET~2_IPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Data Science Project_2</font>
# <font color='blue'>Machine Learning e IA em Ambientes Distribuídos</font>

# Projeto - Parte 2 - Feature Engineering e Rolling Compute

In [ ]:
# Imports
import sys
import os
import re
import time
import subprocess

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.types import *
from pyspark.sql.dataframe import *
from pyspark.sql.functions import col,udf,lag,date_add,explode,lit,concat,unix_timestamp
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from datetime import datetime, timedelta
from pyspark.sql import Row

import pyspark.sql.functions as F

start_time = time.time()

## Definindo a Lista de Recursos (Atributos)

In [ ]:
rolling_features = [
    'warn_type1_total', 'warn_type2_total', 
    'pca_1_warn','pca_2_warn', 'pca_3_warn', 'pca_4_warn', 'pca_5_warn',
    'pca_6_warn','pca_7_warn', 'pca_8_warn', 'pca_9_warn', 'pca_10_warn',
    'pca_11_warn','pca_12_warn', 'pca_13_warn', 'pca_14_warn', 'pca_15_warn',
    'pca_16_warn','pca_17_warn', 'pca_18_warn', 'pca_19_warn', 'pca_20_warn',
    'problem_type_1', 'problem_type_2', 'problem_type_3','problem_type_4',
    'problem_type_1_per_usage1','problem_type_2_per_usage1',
    'problem_type_3_per_usage1','problem_type_4_per_usage1',
    'problem_type_1_per_usage2','problem_type_2_per_usage2',
    'problem_type_3_per_usage2','problem_type_4_per_usage2',                
    'fault_code_type_1_count', 'fault_code_type_2_count', 'fault_code_type_3_count', 'fault_code_type_4_count',                          
    'fault_code_type_1_count_per_usage1','fault_code_type_2_count_per_usage1',
    'fault_code_type_3_count_per_usage1', 'fault_code_type_4_count_per_usage1',
    'fault_code_type_1_count_per_usage2','fault_code_type_2_count_per_usage2',
    'fault_code_type_3_count_per_usage2', 'fault_code_type_4_count_per_usage2']
               
# Lag window 3, 7, 14, 30, 90 dias
lags = [3, 7, 14, 30, 90]

print(len(rolling_features))

46


## Avaliando a Performance do Hardware

- Antes do cálculo contínuo, salve o trabalho original como um arquivo parquet no Notebook 1. Isso acelerará todo o processo porque não precisamos repetir todos os passos anteriores. Isso também ajudará a reduzir a linhagem.
- Imprima o lag e o nome do recurso para acompanhar o progresso.
- Use o comando "htop" no terminal para monitorar quantas CPUs estão sendo executadas para uma tarefa específica. Para o rolling compute, consideramos duas abordagens possíveis: 1) Use o cluster Spark para executar o cálculo contínuo em paralelo; 2) Use um único nó Spark em uma VM poderosa. Ao observar o painel do htop, vimos que todos os núcleos estavam sendo executados ao mesmo tempo para uma única tarefa. Portanto, se dissermos que dividimos a carga de trabalho em vários nós e cada nó executa um tipo de cálculo contínuo, a quantidade de tempo gasto será comparável à execução de tudo de maneira sequencial em um único nó, ou seja, Spark em uma máquina poderosa.
- Use "%% time" para cada célula para obter uma estimativa do tempo total de execução, então teremos uma ideia melhor de onde e o que otimizar no processo.
- Materializar os resultados intermediários, armazenando em cache na memória ou gravando como arquivos parquet. Optamos por salvar como arquivos parquet porque não queríamos repetir a computação novamente caso o cache não funcionasse ou qualquer parte da computação não funcionasse.
- Por que parquet? Há muitas razões, só para citar algumas: o parquet não só salva os dados, mas também o esquema, é um formato de arquivo preferido pelo Spark, você tem permissão para ler apenas os dados de que precisa, etc.

<br>


## Rolling Compute
### Rolling Mean

In [ ]:
%%time

# Carregando o resultado do notebook 1
df = sqlContext.read.parquet('notebook1_resultado.parquet')

for lag_n in lags:
    wSpec = Window.partitionBy('deviceid').orderBy('date').rowsBetween(1 - lag_n, 0)
    for col_name in rolling_features:
        df = df.withColumn(col_name+'_rollingmean_'+str(lag_n), F.avg(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

# Salvando o resultado intermediário
df.write.mode('overwrite').parquet('data_rollingmean.parquet')


Lag = 3, Column = warn_type1_total
Lag = 3, Column = warn_type2_total
Lag = 3, Column = pca_1_warn
Lag = 3, Column = pca_2_warn
Lag = 3, Column = pca_3_warn
Lag = 3, Column = pca_4_warn
Lag = 3, Column = pca_5_warn
Lag = 3, Column = pca_6_warn
Lag = 3, Column = pca_7_warn
Lag = 3, Column = pca_8_warn
Lag = 3, Column = pca_9_warn
Lag = 3, Column = pca_10_warn
Lag = 3, Column = pca_11_warn
Lag = 3, Column = pca_12_warn
Lag = 3, Column = pca_13_warn
Lag = 3, Column = pca_14_warn
Lag = 3, Column = pca_15_warn
Lag = 3, Column = pca_16_warn
Lag = 3, Column = pca_17_warn
Lag = 3, Column = pca_18_warn
Lag = 3, Column = pca_19_warn
Lag = 3, Column = pca_20_warn
Lag = 3, Column = problem_type_1
Lag = 3, Column = problem_type_2
Lag = 3, Column = problem_type_3
Lag = 3, Column = problem_type_4
Lag = 3, Column = problem_type_1_per_usage1
Lag = 3, Column = problem_type_2_per_usage1
Lag = 3, Column = problem_type_3_per_usage1
Lag = 3, Column = problem_type_4_per_usage1
Lag = 3, Column = problem_type_

### Rolling Difference

In [ ]:
%%time

# Carregando o resultado do notebook 1
df = sqlContext.read.parquet('notebook1_resultado.parquet')

for lag_n in lags:
    wSpec = Window.partitionBy('deviceid').orderBy('date').rowsBetween(1 - lag_n, 0)
    for col_name in rolling_features:
        df = df.withColumn(col_name+'_rollingdiff_'+str(lag_n), col(col_name) - F.avg(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

rollingdiff = df.select(['key'] + list(s for s in df.columns if "rollingdiff" in s))

# Salvando o resultado intermediário
rollingdiff.write.mode('overwrite').parquet('data_rollingdiff.parquet')


Lag = 3, Column = warn_type1_total
Lag = 3, Column = warn_type2_total
Lag = 3, Column = pca_1_warn
Lag = 3, Column = pca_2_warn
Lag = 3, Column = pca_3_warn
Lag = 3, Column = pca_4_warn
Lag = 3, Column = pca_5_warn
Lag = 3, Column = pca_6_warn
Lag = 3, Column = pca_7_warn
Lag = 3, Column = pca_8_warn
Lag = 3, Column = pca_9_warn
Lag = 3, Column = pca_10_warn
Lag = 3, Column = pca_11_warn
Lag = 3, Column = pca_12_warn
Lag = 3, Column = pca_13_warn
Lag = 3, Column = pca_14_warn
Lag = 3, Column = pca_15_warn
Lag = 3, Column = pca_16_warn
Lag = 3, Column = pca_17_warn
Lag = 3, Column = pca_18_warn
Lag = 3, Column = pca_19_warn
Lag = 3, Column = pca_20_warn
Lag = 3, Column = problem_type_1
Lag = 3, Column = problem_type_2
Lag = 3, Column = problem_type_3
Lag = 3, Column = problem_type_4
Lag = 3, Column = problem_type_1_per_usage1
Lag = 3, Column = problem_type_2_per_usage1
Lag = 3, Column = problem_type_3_per_usage1
Lag = 3, Column = problem_type_4_per_usage1
Lag = 3, Column = problem_type_

### Rolling Std

In [ ]:
%%time

# Carregando o resultado do notebook 1
df = sqlContext.read.parquet('notebook1_resultado.parquet')

for lag_n in lags:
    wSpec = Window.partitionBy('deviceid').orderBy('date').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        df = df.withColumn(col_name+'_rollingstd_'+str(lag_n), F.stddev(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

# Existem alguns missing ausentes para recursos do rollingstd
rollingstd_features = list(s for s in df.columns if "rollingstd" in s)
df = df.fillna(0, subset=rollingstd_features)
rollingstd = df.select(['key'] + list(s for s in df.columns if "rollingstd" in s))

# Salvando o resultado intermediário
rollingstd.write.mode('overwrite').parquet('data_rollingstd.parquet')

Lag = 3, Column = warn_type1_total
Lag = 3, Column = warn_type2_total
Lag = 3, Column = pca_1_warn
Lag = 3, Column = pca_2_warn
Lag = 3, Column = pca_3_warn
Lag = 3, Column = pca_4_warn
Lag = 3, Column = pca_5_warn
Lag = 3, Column = pca_6_warn
Lag = 3, Column = pca_7_warn
Lag = 3, Column = pca_8_warn
Lag = 3, Column = pca_9_warn
Lag = 3, Column = pca_10_warn
Lag = 3, Column = pca_11_warn
Lag = 3, Column = pca_12_warn
Lag = 3, Column = pca_13_warn
Lag = 3, Column = pca_14_warn
Lag = 3, Column = pca_15_warn
Lag = 3, Column = pca_16_warn
Lag = 3, Column = pca_17_warn
Lag = 3, Column = pca_18_warn
Lag = 3, Column = pca_19_warn
Lag = 3, Column = pca_20_warn
Lag = 3, Column = problem_type_1
Lag = 3, Column = problem_type_2
Lag = 3, Column = problem_type_3
Lag = 3, Column = problem_type_4
Lag = 3, Column = problem_type_1_per_usage1
Lag = 3, Column = problem_type_2_per_usage1
Lag = 3, Column = problem_type_3_per_usage1
Lag = 3, Column = problem_type_4_per_usage1
Lag = 3, Column = problem_type_

### Rolling Max

In [ ]:
%%time

# Carregando o resultado do notebook 1
df = sqlContext.read.parquet('notebook1_resultado.parquet')

for lag_n in lags:
    wSpec = Window.partitionBy('deviceid').orderBy('date').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        df = df.withColumn(col_name+'_rollingmax_'+str(lag_n), F.max(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

rollingmax = df.select(['key'] + list(s for s in df.columns if "rollingmax" in s))

# Salvando o resultado intermediário
rollingmax.write.mode('overwrite').parquet('data_rollingmax.parquet')

Lag = 3, Column = warn_type1_total
Lag = 3, Column = warn_type2_total
Lag = 3, Column = pca_1_warn
Lag = 3, Column = pca_2_warn
Lag = 3, Column = pca_3_warn
Lag = 3, Column = pca_4_warn
Lag = 3, Column = pca_5_warn
Lag = 3, Column = pca_6_warn
Lag = 3, Column = pca_7_warn
Lag = 3, Column = pca_8_warn
Lag = 3, Column = pca_9_warn
Lag = 3, Column = pca_10_warn
Lag = 3, Column = pca_11_warn
Lag = 3, Column = pca_12_warn
Lag = 3, Column = pca_13_warn
Lag = 3, Column = pca_14_warn
Lag = 3, Column = pca_15_warn
Lag = 3, Column = pca_16_warn
Lag = 3, Column = pca_17_warn
Lag = 3, Column = pca_18_warn
Lag = 3, Column = pca_19_warn
Lag = 3, Column = pca_20_warn
Lag = 3, Column = problem_type_1
Lag = 3, Column = problem_type_2
Lag = 3, Column = problem_type_3
Lag = 3, Column = problem_type_4
Lag = 3, Column = problem_type_1_per_usage1
Lag = 3, Column = problem_type_2_per_usage1
Lag = 3, Column = problem_type_3_per_usage1
Lag = 3, Column = problem_type_4_per_usage1
Lag = 3, Column = problem_type_

### Rolling Min

In [ ]:
%%time

# Carregando o resultado do notebook 1
df = sqlContext.read.parquet('notebook1_resultado.parquet')

for lag_n in lags:
    wSpec = Window.partitionBy('deviceid').orderBy('date').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        df = df.withColumn(col_name+'_rollingmin_'+str(lag_n), F.min(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

rollingmin = df.select(['key'] + list(s for s in df.columns if "rollingmin" in s))

# Salvando o resultado intermediário
rollingmin.write.mode('overwrite').parquet('data_rollingmin.parquet')

Lag = 3, Column = warn_type1_total
Lag = 3, Column = warn_type2_total
Lag = 3, Column = pca_1_warn
Lag = 3, Column = pca_2_warn
Lag = 3, Column = pca_3_warn
Lag = 3, Column = pca_4_warn
Lag = 3, Column = pca_5_warn
Lag = 3, Column = pca_6_warn
Lag = 3, Column = pca_7_warn
Lag = 3, Column = pca_8_warn
Lag = 3, Column = pca_9_warn
Lag = 3, Column = pca_10_warn
Lag = 3, Column = pca_11_warn
Lag = 3, Column = pca_12_warn
Lag = 3, Column = pca_13_warn
Lag = 3, Column = pca_14_warn
Lag = 3, Column = pca_15_warn
Lag = 3, Column = pca_16_warn
Lag = 3, Column = pca_17_warn
Lag = 3, Column = pca_18_warn
Lag = 3, Column = pca_19_warn
Lag = 3, Column = pca_20_warn
Lag = 3, Column = problem_type_1
Lag = 3, Column = problem_type_2
Lag = 3, Column = problem_type_3
Lag = 3, Column = problem_type_4
Lag = 3, Column = problem_type_1_per_usage1
Lag = 3, Column = problem_type_2_per_usage1
Lag = 3, Column = problem_type_3_per_usage1
Lag = 3, Column = problem_type_4_per_usage1
Lag = 3, Column = problem_type_

## Unindo o resultado das 5 células de computação anteriores:

- Join in Spark geralmente é muito lento, é melhor reduzir o número de partições antes da junção.
- Verifique o número de partições do dataframe do pyspark.
- ** repartition vs coalesce **. Se quisermos apenas reduzir o número de partições, é melhor usar a coalesce porque a repartição envolve a reorganização, que é computacionalmente mais cara e leva mais tempo.
<br>


In [ ]:
# Import dos resultados parciais
rollingmean = sqlContext.read.parquet('data_rollingmean.parquet')
rollingdiff = sqlContext.read.parquet('data_rollingdiff.parquet')
rollingstd = sqlContext.read.parquet('data_rollingstd.parquet')
rollingmax = sqlContext.read.parquet('data_rollingmax.parquet')
rollingmin = sqlContext.read.parquet('data_rollingmin.parquet')

# Verifique o número de partições para cada conjunto de dados
print(rollingmean.rdd.getNumPartitions())
print(rollingdiff.rdd.getNumPartitions())
print(rollingstd.rdd.getNumPartitions())
print(rollingmax.rdd.getNumPartitions())
print(rollingmin.rdd.getNumPartitions())

33
33
33
31
31


In [ ]:
%%time

# Para tornar a junção mais rápida, reduza o número de partições (não necessariamente para "1")
rollingmean = rollingmean.coalesce(1)
rollingdiff = rollingdiff.coalesce(1)
rollingstd = rollingstd.coalesce(1)
rollingmax = rollingmax.coalesce(1)
rollingmin = rollingmin.coalesce(1)

rolling_result = rollingmean.join(rollingdiff, 'key', 'inner')\
                 .join(rollingstd, 'key', 'inner')\
                 .join(rollingmax, 'key', 'inner')\
                 .join(rollingmin, 'key', 'inner')
            

## Grave o resultado final como arquivo parquet para o trabalho no Notebook_3
rolling_result.write.mode('overwrite').parquet('notebook2_resultado.parquet')

CPU times: user 901 ms, sys: 303 ms, total: 1.2 s
Wall time: 1h 50min 38s


# Fim - Parte 2